## Upload Data

In [1]:
import pandas as pd
df = pd.read_csv('G:\\My Drive\\Tugas SEM3\\Kecerdasan Artifisial\\Project AI\\datajadi.csv')
# df1 = pd.read_csv('G:\\My Drive\\Tugas SEM3\\Pemrosesan Teks\\Project Pemro\\data_resample1.csv')

In [2]:
display(df.head())

,judulpluskonten,preproses_text,Number of Determinants,Number of Short Sentence,Number of Long Sentence,smog_index,Subjectivity,Number of Syllables,Rate Adjective and Adverb,Number of Sentences,...,number of special karakter,number of capital letter,gunning_fog_index,ari,number of word,word per sentence,number of verb,number of adjective,label,vec
0,Health insurers quietly shape Obamacare replac...,health insur quietli shape obamacar replac few...,4,3,38,15.0,0.395471,1614,58.0,148,...,124,150,13.43,19.987545,1006,30.216216,136,83,0,[ 0.02840978 0.00446486 -0.00950904 0.032875...
1,Migrants Refuse To Leave Train At Refugee Camp...,migrant refus leav train refuge camp hungarymi...,4,2,18,11.1,0.289697,753,28.0,82,...,85,115,10.67,15.822994,486,25.545455,67,34,0,[ 0.02864144 0.004986 -0.01179579 0.033683...
2,Manafort meets with Senate intelligence panel:...,manafo meet senat intellig panel spokesmanwash...,4,1,3,15.9,0.222222,166,2.5,12,...,18,37,15.92,23.579263,95,37.666667,13,3,0,[ 3.06978021e-02 4.67474674e-05 -1.02226874e-...
3,CBO Releases Score of Paul Ryan’s American Hea...,cbo releas score paul ryan american health car...,4,0,2,0.0,0.210000,84,2.5,8,...,17,55,16.27,21.590125,63,40.000000,5,3,0,[ 0.03636537 -0.00171204 -0.01732128 0.036230...
4,Conservatives Urge Sessions to Clean Out Obama...,conserv urg session clean obama civil right di...,4,1,13,18.0,0.366094,743,18.5,52,...,64,97,18.41,23.542137,409,36.846154,45,28,0,[ 0.02952077 0.00713297 -0.01201566 0.031389...


## import library yang dibutuhkan

In [3]:
import numpy as np
from random import randint
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split


In [25]:
classifiers = ['LinearSVM', 'RadialSVM', 
               'Logistic',  'RandomForest', 
               'AdaBoost',  'DecisionTree', 
               'KNeighbors','GradientBoosting']

models = [svm.SVC(kernel='linear'),
          svm.SVC(kernel='rbf'),
          LogisticRegression(max_iter = 1000),
          RandomForestClassifier(n_estimators=200, random_state=0),
          AdaBoostClassifier(random_state = 0),
          DecisionTreeClassifier(random_state=0),
          KNeighborsClassifier(),
          GradientBoostingClassifier(random_state=0)]


def acc_score(X_train,X_test,y_train,y_test):
    Score = pd.DataFrame({"Classifier":classifiers})
    j = 0
    acc = []
    for i in models:
        model = i
        model.fit(X_train,y_train)
        predictions = model.predict(X_test)
        acc.append(accuracy_score(y_test,predictions))
        j = j+1     
    Score["Accuracy"] = acc
    Score.sort_values(by="Accuracy", ascending=False,inplace = True)
    Score.reset_index(drop=True, inplace=True)
    return Score
    

In [21]:
def initialize_population(num_individuals, num_features):
    return np.random.choice([0, 1], size=(num_individuals, num_features))

def fitness(individual, X_train, X_test, y_train, y_test):
    selected_features = X_train.columns[individual == 1]
    clf = RandomForestClassifier(n_estimators=200, random_state=0)
    clf.fit(X_train[selected_features], y_train)
    y_pred = clf.predict(X_test[selected_features])
    return accuracy_score(y_test, y_pred)

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1) - 1)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(individual, mutation_rate):
    mutation_mask = np.random.rand(len(individual)) < mutation_rate
    individual[mutation_mask] = 1 - individual[mutation_mask] 
    return individual

def genetic_algorithm(X_train, X_test, y_train, y_test, num_generations=5, population_size=5, crossover_rate=0.8, mutation_rate=0.01):
    num_individuals, num_features = population_size, X_train.shape[1]
    population = initialize_population(num_individuals, num_features)
    
    seleksi_fiturpergenerasi = []
    for generation in range(num_generations):
        
        #evaluasi fittest tiap gen
        fitness_scores = [fitness(individual, X_train, X_test, y_train, y_test) for individual in population]

        selected_indices = np.argsort(fitness_scores)[::-1][:int(crossover_rate * population_size)]
        selected_population = population[selected_indices]

        #crossover
        new_population = []
        for _ in range(num_individuals // 2):
            idx_parent1, idx_parent2 = np.random.choice(len(selected_population), size=2, replace=False)
            parent1, parent2 = selected_population[idx_parent1], selected_population[idx_parent2]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])

        #mutasi gen
        new_population = [mutate(individual, mutation_rate) for individual in new_population]

        #menggantikan populasi lama dgn baru
        population[:len(new_population)] = new_population

        #print hasil terbaik tiap generasi
        best_fitness = np.max(fitness_scores)
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness:.4f}")
        seleksi_fiturpergenerasi.append([np.where(individual == 1)[0] for individual in population])

    best_individual = population[np.argmax(fitness_scores)]

    # Cetak atau tampilkan informasi fitur yang dipilih pada setiap generasi
    for gen, selected_features in enumerate(seleksi_fiturpergenerasi):
        print(f"Generation {gen + 1}: Selected features for each individual: {selected_features}")


    return best_individual

 
X = df.drop(columns=['judulpluskonten','preproses_text','label','vec'])
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

best_individual = genetic_algorithm(X_train, X_test, y_train, y_test,num_generations=5,population_size=10)



Generation 1, Best Fitness: 0.8578
Generation 2, Best Fitness: 0.8508
Generation 3, Best Fitness: 0.8528
Generation 4, Best Fitness: 0.8558
Generation 5, Best Fitness: 0.8558
Generation 1: Selected features for each individual: [array([ 1,  5,  6,  7, 10, 14, 16], dtype=int64), array([ 2,  4,  6,  8,  9, 11, 13, 14, 15, 16], dtype=int64), array([ 0,  1,  2,  3,  4,  6,  8,  9, 11, 13, 14, 15, 16], dtype=int64), array([ 1,  5,  6,  9, 10, 11, 12], dtype=int64), array([ 0,  1,  2,  3,  4,  5,  6,  8, 14, 15, 16], dtype=int64), array([ 1,  3,  5,  6,  9, 10, 11, 12], dtype=int64), array([ 1,  3,  4,  7,  9, 10, 11, 12, 14, 15, 16], dtype=int64), array([ 0,  2,  3,  4,  5,  6,  8, 10, 12, 13, 14], dtype=int64), array([ 1,  3,  4,  7,  8,  9, 12, 14, 16], dtype=int64), array([ 2,  3,  4,  5,  6,  8, 14, 15, 16], dtype=int64)]
Generation 2: Selected features for each individual: [array([ 1,  5,  6,  9, 10, 11, 12], dtype=int64), array([ 1,  3,  4,  7,  8,  9, 12, 14, 16], dtype=int64), array

In [26]:
X = df.drop(columns=['judulpluskonten','preproses_text','label','vec'])
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

score1 = acc_score(X_train, X_test, y_train, y_test)
score1

,Classifier,Accuracy
0,RandomForest,0.87075
1,GradientBoosting,0.84550
2,LinearSVM,0.82575
3,Logistic,0.81775
4,AdaBoost,0.81725
5,KNeighbors,0.81175
6,DecisionTree,0.79150
7,RadialSVM,0.78525


In [22]:
print(best_individual)

[1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 1]


In [14]:
print(best_individual)

[1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1]


In [24]:
df_seleksi = df[['Number of Determinants',
       'Number of Short Sentence', 'Number of Long Sentence', 'smog_index',
       'Subjectivity','Rate Adjective and Adverb','number of special karakter',
       'number of capital letter', 'gunning_fog_index', 'ari',
       'number of word','number of verb',
       'number of adjective','label']]
df_seleksi.shape

(20000, 14)

In [ ]:
df_seleksi.to_csv('G:\My Drive\Tugas SEM3\Kecerdasan Artifisial\Project AI\df_seleksi.csv',index=False)